In [1]:
%run ../00_default_options.ipynb

In [2]:
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

In [3]:
cohort = Cohort.load(id=1)

In [4]:
onset_dataframe = OnsetDataFrame.load(id=17)

In [5]:
feature_pipeline = FeaturePipeline.load(id=20)

In [6]:
import morpher
from morpher.jobs import *

Using TensorFlow backend.


In [7]:
regexes = [
    'any__Diagnosis__', 
    #'any__Procedure__', 
    #'any__Drug__', 
    #'any__Material__', 
    #'any__Encounter__', 
    #'any__AlcoholUse__', 
    #'any__DrugUse__', 
    #'any__TobaccoUse__',
    'min__Height__', 
    'min__Weight__', 
    'min__VitalSign__', 
    #'min__Measurement__', 
    'min__LabValue__', 
    'max__Height__', 
    'max__Weight__', 
    'max__VitalSign__', 
    #'max__Measurement__', 
    'max__LabValue__'
]

agg_func_regex = ""
for r in regexes:
    agg_func_regex += r + "|"
agg_func_regex = agg_func_regex[:-1]

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]] = label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]].fillna(False)
    
    scaled_data, _ = Scale().execute(data=label_encoded_data, target=target)
    
    imputed_data, _ = Impute().execute(data=scaled_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [8]:
from itertools import product

In [9]:
thresholds_numeric = [(x / 100) for x in range(5, 100, 5)]
observation_windows_numeric = [[x, -1] for x in range(-361, -1, 30)]
thresholds_occurring = [(x / 100) for x in range(5, 10, 5)]
observation_windows_occurring = [[x, -1] for x in range(-361, -331, 30)]

In [10]:
iterator = product(
    thresholds_numeric,
    observation_windows_numeric,
    thresholds_occurring,
    observation_windows_occurring
)

configs = [x for x in iterator]
targets = ['urinary_tract_infection_onset_from_0_days_after_to_365_days_after']
algorithms = [morpher.config.algorithms.GBDT, morpher.config.algorithms.RF, morpher.config.algorithms.DT, morpher.config.algorithms.LR]
samplers = samplers = [morpher.config.samplers.RANDOM, morpher.config.samplers.URANDOM, morpher.config.samplers.SMOTE, morpher.config.samplers.NOSAMPLER]

In [11]:
pipeline = training.execute(
    comment='baseline-3 UTI with RFE - focus numeric values',
    version='13.1.0',
    cohort=cohort,
    onset_dataframe=onset_dataframe,
    feature_pipeline=feature_pipeline,
    data_loader=data_loader,
    observation_iterator=configs,
    targets=targets,
    algorithms=algorithms,
    samplers=samplers,
    feature_type_occurring="occurring",
    feature_type_numeric="numeric",
    rfe__run=True,
    rfe__step_size=50
)

Columns (11,18,19,20,24,35,64,83,90,92,94,96,97,106,114,117,120,125,126,133,136,140,141,142,143,144,150,178,235,244,248,288,311,317,320,325,326,330,334,341,353,359,360,361,365,402,421,425,430,434,436,439,446) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7fb98124f978>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


Fitting estimator with 500 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 14.674ms

Fitting estimator with 450 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 28.91ms

Fitting estimator with 400 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 689.273ms

Fitting estimator with 350 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 206.549ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 881.295ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 746.471ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 208.579ms

*** Training of model 

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 108.697ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 114.771ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 84.156ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 188.603ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 70.289ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 70.967ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 70.422ms

Fitting estimator with 500 features.
*** Training of model 'LogisticRegression' started.


...

Columns (11,18,19,20,24,35,64,83,90,92,94,96,97,106,114,117,120,125,126,133,136,140,141,142,143,144,150,178,235,244,248,288,311,317,320,325,326,330,334,341,353,359,360,361,365,402,421,425,430,434,436,439,446) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7fb9804ee0b8>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


Fitting estimator with 130 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 957.197ms

Fitting estimator with 80 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 696.302ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 346.264ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 345.607ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 419.524ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 389.856ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 391.668ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
pipeline.id

242